In [ ]:
import os
import shutil
import random

# 원본 데이터가 있는 폴더 경로
source_dir = 'IMG'
# 새로운 데이터 구조를 만들 폴더 경로
target_dir = 'structured_data'

# train, val, test 비율 설정
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# 대상 폴더 생성
os.makedirs(os.path.join(target_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(target_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(target_dir, 'test'), exist_ok=True)

# 각 클래스 폴더를 순회
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    
    if os.path.isdir(class_path):
        # 이미지 파일 목록 가져오기
        images = [img for img in os.listdir(class_path) if img.endswith(('.png', '.jpg', '.jpeg'))]
        random.shuffle(images)  # 이미지 랜덤화
        
        # 데이터 분할
        total_images = len(images)
        train_count = int(total_images * train_ratio)
        val_count = int(total_images * val_ratio)
        
        # train, val, test 이미지 분배
        train_images = images[:train_count]
        val_images = images[train_count:train_count + val_count]
        test_images = images[train_count + val_count:]

        # 클래스별 하위 폴더 생성
        os.makedirs(os.path.join(target_dir, 'train', class_name), exist_ok=True)
        os.makedirs(os.path.join(target_dir, 'val', class_name), exist_ok=True)
        os.makedirs(os.path.join(target_dir, 'test', class_name), exist_ok=True)

        # 각 이미지를 대상 폴더로 이동
        for img in train_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(target_dir, 'train', class_name, img))
        for img in val_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(target_dir, 'val', class_name, img))
        for img in test_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(target_dir, 'test', class_name, img))

print("폴더 구조가 성공적으로 생성되었습니다.")


In [ ]:
import os
import shutil
import random
from PIL import Image, ImageEnhance
import numpy as np

# 이미지 증강 함수 정의
def augment_image(image):
    # 회전
    angle = random.randint(-30, 30)
    image = image.rotate(angle)

    # 랜덤 크롭
    width, height = image.size
    left = random.randint(0, int(width * 0.1))
    top = random.randint(0, int(height * 0.1))
    right = random.randint(int(width * 0.9), width)
    bottom = random.randint(int(height * 0.9), height)
    image = image.crop((left, top, right, bottom))

    # 좌우 반전
    if random.random() > 0.5:
        image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # 색상 변화
    if random.random() > 0.5:
        enhancer = ImageEnhance.Color(image)
        factor = random.uniform(0.5, 1.5)  # 색상 강도를 조절
        image = enhancer.enhance(factor)

    # 밝기 조절
    if random.random() > 0.5:
        enhancer = ImageEnhance.Brightness(image)
        factor = random.uniform(0.5, 1.5)  # 밝기 강도를 조절
        image = enhancer.enhance(factor)

    # 노이즈 추가
    if random.random() > 0.5:
        noise = np.random.normal(0, 25, (image.size[1], image.size[0], 3)).astype(np.uint8)
        image_array = np.array(image)
        noisy_image = np.clip(image_array + noise, 0, 255).astype(np.uint8)
        image = Image.fromarray(noisy_image)

    return image

# 원본 데이터가 있는 폴더 경로
source_dir = 'IMG'
# 새로운 데이터 구조를 만들 폴더 경로
target_dir = 'structured_data'

# train, val, test 비율 설정
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# 대상 폴더 생성
os.makedirs(os.path.join(target_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(target_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(target_dir, 'test'), exist_ok=True)

# 각 클래스 폴더를 순회
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    
    if os.path.isdir(class_path):
        # 이미지 파일 목록 가져오기
        images = [img for img in os.listdir(class_path) if img.endswith(('.png', '.jpg', '.jpeg'))]
        augmented_images = []  # 증강된 이미지 저장 리스트

        # 각 이미지에 대해 증강 적용
        for img in images:
            img_path = os.path.join(class_path, img)
            image = Image.open(img_path)
            augmented_image = augment_image(image)
            augmented_images.append(augmented_image)

        # 랜덤화
        random.shuffle(augmented_images)

        # 데이터 분할
        total_images = len(augmented_images)
        train_count = int(total_images * train_ratio)
        val_count = int(total_images * val_ratio)
        
        # train, val, test 이미지 분배
        train_images = augmented_images[:train_count]
        val_images = augmented_images[train_count:train_count + val_count]
        test_images = augmented_images[train_count + val_count:]

        # 클래스별 하위 폴더 생성
        os.makedirs(os.path.join(target_dir, 'train', class_name), exist_ok=True)
        os.makedirs(os.path.join(target_dir, 'val', class_name), exist_ok=True)
        os.makedirs(os.path.join(target_dir, 'test', class_name), exist_ok=True)

        # 각 이미지를 대상 폴더로 저장
        for idx, img in enumerate(train_images):
            img.save(os.path.join(target_dir, 'train', class_name, f'augmented_{idx}.jpg'))
        for idx, img in enumerate(val_images):
            img.save(os.path.join(target_dir, 'val', class_name, f'augmented_{idx}.jpg'))
        for idx, img in enumerate(test_images):
            img.save(os.path.join(target_dir, 'test', class_name, f'augmented_{idx}.jpg'))

print("다양한 이미지 증강 후 랜덤으로 분배가 완료되었습니다.")


In [ ]:
import os
import random
from PIL import Image, ImageEnhance
import numpy as np

# 이미지 증강 함수 정의
def augment_image(image):
    # 회전
    angle = random.randint(-30, 30)
    image = image.rotate(angle)

    # 크기 조정
    scale = random.uniform(0.8, 1.2)
    new_size = (int(image.size[0] * scale), int(image.size[1] * scale))
    image = image.resize(new_size)

    # 랜덤 크롭
    width, height = image.size
    left = random.randint(0, int(width * 0.1))
    top = random.randint(0, int(height * 0.1))
    right = random.randint(int(width * 0.9), width)
    bottom = random.randint(int(height * 0.9), height)
    image = image.crop((left, top, right, bottom))

    # 좌우 반전
    if random.random() > 0.5:
        image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # 노이즈 추가
    if random.random() > 0.5:
        noise = np.random.normal(0, 25, (image.size[1], image.size[0])).astype(np.uint8)
        image_array = np.array(image)
        noisy_image = np.clip(image_array + noise, 0, 255).astype(np.uint8)
        image = Image.fromarray(noisy_image)

    # 밝기 조절
    if random.random() > 0.5:
        enhancer = ImageEnhance.Brightness(image)
        factor = random.uniform(0.5, 1.5)  # 밝기 강도를 조절
        image = enhancer.enhance(factor)

    # 대비 조절
    if random.random() > 0.5:
        enhancer = ImageEnhance.Contrast(image)
        factor = random.uniform(0.5, 1.5)  # 대비 강도를 조절
        image = enhancer.enhance(factor)

    return image

# 원본 데이터가 있는 폴더 경로
source_dir = 'black_and_white_images'
# 새로운 데이터 구조를 만들 폴더 경로
target_dir = 'augmented_images'

# 대상 폴더 생성
os.makedirs(target_dir, exist_ok=True)

# 각 이미지 파일을 순회하며 증강 수행
for img_name in os.listdir(source_dir):
    if img_name.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(source_dir, img_name)
        image = Image.open(img_path).convert('L')  # 흑백으로 변환
        augmented_image = augment_image(image)
        
        # 증강된 이미지 저장
        augmented_image.save(os.path.join(target_dir, f'augmented_{img_name}'))

print("흑백 이미지 증강이 완료되었습니다.")
